In [1]:
import pandas as pd
import numpy as np

In [2]:
# sheet_name=None으로 불러오면 엑셀 파일의 모든 시트를 딕셔너리 형태로 불러온다.
# 키: sheetname, 값: 데이터프레임
df_01_to_07 = pd.read_excel("../data/reefer_data_230126.xlsx", sheet_name=None) 
df_08_and_12 = pd.read_excel("../data/reefer_data_8월_12월.xlsx", sheet_name=None)

In [3]:
# sheet 목록 출력
df_01_to_07.keys(), df_08_and_12.keys()

(dict_keys(['1월', '2월', '3월', '4월', '5월', '6월', '7월', 'ALARM CODE', '상태정보(OPERATING MODE)']),
 dict_keys(['2022_08', '2022_12', 'ALARM CODE', '상태정보(OPERATING MODE)']))

In [4]:
def split_made_cd(df):
    """
    데이터를 제조사에 따라 나누는 함수입니다.

    Input
    -----
    df: 데이터프레임

    Returns
    -------
    carrier, daikin: 나누어진 데이터프레임(튜플)
    """
    carrier = df.loc[df['made_cd'] == 'CARRIER']
    daikin = df.loc[df['made_cd'] == 'DAIKIN']

    return carrier, daikin

In [5]:
def kelvin_to_celsius(df):
    """
    데이터의 온도 관련 변수들을 섭씨온도로 변환하는 함수입니다.

    Input
    -----
    df: 온도 관련 변수들이 있는 데이터프레임

    Return
    ------
    df_celsius: 섭씨온도로 변환된 데이터프레임
    """
    df_celsius = df.copy()

    K = 273.15
    
    df_celsius['ambient_temperature'] = df['ambient_temperature'] - K
    df_celsius['return_air_temperature'] = df['return_air_temperature'] - K
    df_celsius['supply_air_temperature'] = df['supply_air_temperature'] - K
    df_celsius['temperature_setpoint'] = df['temperature_setpoint'] - K

    return df_celsius

In [6]:
def concat_dataframe(loaded_excel):
    """
    sheet가 여러 개인 데이터를 하나로 합치는 함수입니다.

    Input
    -----
    loaded_excel: sheet_name=None으로 불러온 딕셔너리

    Return
    ------
    concated_df: 하나로 합쳐진 데이터프레임
    """ 
    df_list = list(loaded_excel.keys())[:-2] # 알람코드와 상태정보는 삭제

    concat_list = []
    for key in df_list:
        df = kelvin_to_celsius(loaded_excel[key])
        concat_list.append(df)

    concated_df = pd.concat(concat_list)

    return concated_df

In [7]:
df_concat_01_to_07 = concat_dataframe(df_01_to_07)
df_concat_08_and_12 = concat_dataframe(df_08_and_12)

In [8]:
df_carrier_JAN_to_JUL, _ = split_made_cd(df_concat_01_to_07) # DAIKIN 데이터는 받지 않음
df_carrier_AUG_and_DEC, _ = split_made_cd(df_concat_08_and_12) # DAIKIN 데이터는 받지 않음

In [9]:
df_carrier_JAN_to_JUL

,alarm_codes,ambient_temperature,made_cd,operating_mode,operating_mode_str,reefer_id,return_air_temperature,supply_air_temperature,temperature_setpoint,when_created
1,NaN,-8.29,CARRIER,6.0,IDLE,RDMU5216890,2.92,3.09,3.0,2022-01-01 00:00:23
3,"[""AL60""]",3.90,CARRIER,3.0,COOL,RJCU6995820,-19.83,-20.47,-20.0,2022-01-01 00:01:08
4,NaN,-1.06,CARRIER,6.0,IDLE,CTEU5865614,-24.43,-25.05,-24.0,2022-01-01 00:01:18
6,"[""dAL86""]",9.62,CARRIER,3.0,COOL,SDCU6045746,-17.57,-19.11,-20.0,2022-01-01 00:01:26
7,NaN,-6.06,CARRIER,3.0,COOL,UKFU6720496,-28.10,-30.61,-30.0,2022-01-01 00:01:32
...,...,...,...,...,...,...,...,...,...,...
224838,"[""AL06""]",30.16,CARRIER,3.0,COOL,TRIU6614101,5.16,4.63,4.5,2022-07-31 23:57:55
224840,NaN,25.95,CARRIER,3.0,COOL,SMCU6901080,-3.88,-4.97,-5.0,2022-07-31 23:58:15
224841,NaN,28.09,CARRIER,6.0,IDLE,CTEU3170650,-19.84,-20.03,-20.0,2022-07-31 23:58:24
224845,NaN,26.53,CARRIER,3.0,COOL,MCLU8605448,-22.21,-26.74,-27.5,2022-07-31 23:59:36


In [10]:
df_carrier_AUG_and_DEC

,alarm_codes,ambient_temperature,made_cd,operating_mode,operating_mode_str,reefer_id,return_air_temperature,supply_air_temperature,temperature_setpoint,when_created
2,NaN,28.27,CARRIER,3.0,COOL,ARAU2100036,15.93,15.38,9.5,2022-08-01 00:00:24
3,NaN,25.71,CARRIER,3.0,COOL,CRXU5259163,-19.57,-19.42,-20.0,2022-08-01 00:00:28
5,NaN,27.57,CARRIER,6.0,IDLE,IKRU2200681,-11.55,-14.25,-11.0,2022-08-01 00:00:44
8,NaN,25.07,CARRIER,6.0,IDLE,CRXU5259245,-21.04,-20.30,-21.0,2022-08-01 00:00:58
10,NaN,26.40,CARRIER,6.0,IDLE,CTEU5865614,5.59,6.28,5.0,2022-08-01 00:01:09
...,...,...,...,...,...,...,...,...,...,...
188719,"[""AL51"",""AL52"",""AL53""]",-1.26,CARRIER,4.0,HEAT,TRIU8617244,29.65,31.81,30.0,2022-12-31 23:58:08
188720,NaN,-0.13,CARRIER,6.0,IDLE,BMOU9014938,-27.16,-27.36,-28.0,2022-12-31 23:58:18
188721,NaN,1.06,CARRIER,3.0,COOL,CRXU5259163,-29.79,-29.55,-30.0,2022-12-31 23:58:38
188725,"[""AL53""]",8.59,CARRIER,3.0,COOL,TRIU8431795,-2.12,-2.53,-2.5,2022-12-31 23:59:19


In [11]:
df_carrier_JAN_to_AUG_and_DEC = pd.concat([df_carrier_JAN_to_JUL, df_carrier_AUG_and_DEC]).reset_index(drop=True)

In [12]:
# 데이터 내보내기
df_carrier_JAN_to_AUG_and_DEC.to_csv('../data/carrier_reefer_data_01_to_08_and_12.csv', encoding='cp949', index=False)